# Processing MITGCM data for Mixed-layer depth Analysis 

**Purpose**: Code for computing the mixed layer depth. 

**Luke Colosi | lcolosi@ucsd.edu**

Import python libraries

In [ ]:
import sys
import xarray as xr
from xmitgcm import open_mdsdataset
import numpy as np
import gsw

#--- Other Functions ---# 
sys.path.append("/home/lcolosi/AirSeaCoupling/tools/")
import cartopy_figs as cart

Set data analysis parameters

In [ ]:
# Model parameters 
delta_t = 150  # Time steps of the raw model run (by raw, I mean the time increments that the model is ran at, not time increments that the diagnostics are output at). Units: seconds

# Set time and space parameters  
lat_bnds  = [32.5, 35]                                           # Specifies the latitude bounds for the region to analyze
lon_bnds  = [236.0, 241.0]                                       # Specifies the longitude bounds for the region to analyze
encoding  = {'time': {'units': 'seconds since 2015-12-01 2:00'}} # Specifies the start time of the model run

 
# Set path to project directory
PATH_GRID   = '/data/SO2/SWOT/GRID/BIN/'                    # Space and time grid of the model 
PATH_OUTPUT = '/data/SO2/SWOT/MARA/RUN4_LY/DIAGS_HRLY/'     # Diagnostics of the model
PATH_nc     = '/data/SO3/lcolosi/mitgcm/SWOT_MARA_RUN4_LY/'  # Directory to save netCDFs 
PATH_figs   = '/home/lcolosi/AirSeaCoupling/figs_server/mitgcm/preliminary/'
file_dim    = '3D'                                          # Set the dimension of the data (to include the depth or not)

# Set plotting parameters 
fontsize = 14

Load the grid and diagnostics data into a python structure. The diagnostics that we will be looking at include: 

1. **Potential Temperature** $\theta$: $^\circ C$
2. **Salinity** $S$: $g/kg$
3. **Stratification** $\frac{d\sigma}{dz}$: $kg/m^4$
4. **Zonal, meridional, and vertical velocity components**  $\textbf{u} = (u,v,w)$: $m/s$  

In [ ]:
# Create dataset 
ds = open_mdsdataset(
    PATH_OUTPUT,                    # File path where the model output data is stored (.data and .meta files)
    PATH_GRID,                      # File path to the grid data of the model 
    iters='all',                    # Specifies which iteration of the data to load
    delta_t=delta_t, 
    ignore_unknown_vars=False,      # Specifies whether to ignore any unknown variables that may appear in the dataset
    prefix=['diags_' + file_dim],   # List of prefixes to filter the variables in the dataset
    ref_date="2015-01-01 02:00:00", # Specifies the starting point of the simulation time (which may include the spin up time before diagnostics are output)
    geometry='sphericalpolar',      # Specifies the  grid's geometry is spherical-polar. 
    #chunks={'i': 48, 'i_g': 48, 'j': 56,  'j_g': 56, 'k': 10, 'k_u':10, 'k_l':10, 'k_p1':10} # Chunck data upload into 48x56 pieces (spatial grid is divided into 12 chunks along the x and y axes) chunks={'i': 48, 'i_g': 48, 'j': 56,  'j_g': 56, 'k': 10, 'k_u':10, 'k_l':10, 'k_p1':10, 'time':1} chunks={'XC':48, 'YC':56, 'XG':48, 'YG':56, 'Z':10, 'Zp1':10, 'Zu':10, 'Zl':10, 'time':1}
)

# Convert all variables and coordinates in the dataset to little-endian (the format how multi-byte values are stored into memory)

#--- Variables ---#
for var in ds.data_vars:
    if ds[var].dtype.byteorder == '>' or (ds[var].dtype.byteorder == '=' and sys.byteorder == "big"):  # Check if big-endian
        ds[var] = ds[var].astype(ds[var].dtype.newbyteorder('<'))

#--- Coordinates ---# 
for coord in ds.coords:
    if ds[coord].dtype.byteorder == '>'or (ds[coord].dtype.byteorder == '=' and sys.byteorder == "big"):  # Check if big-endian
        ds[coord] = ds[coord].astype(ds[coord].dtype.newbyteorder('<'))

Slice the array based on latitude and longitude bounds.

In [ ]:
# Slice array
ds_subset = ds.sel(
    XG=slice(*lon_bnds),
    YG=slice(*lat_bnds),
    XC=slice(*lon_bnds),
    YC=slice(*lat_bnds),
)

Compute mixed layer depth for each profile

In [ ]:
import numpy as np
import gsw

def compute_mld_profiles(SA, pt, depth, lat, threshold=0.03, ref_depth=10):
    """
    Compute Mixed Layer Depth (MLD) for multiple profiles using potential density threshold method.

    Parameters
    ----------
    SA : ndarray
        Absolute Salinity [g/kg], shape (n_depths, n_profiles).
    pt : ndarray
        Potential Temperature [°C], shape (n_depths, n_profiles).
    depth : ndarray
        Depth levels [m, positive down], shape (n_depths,).
    lat : float or ndarray
        Latitude(s) [degrees north], scalar or shape (n_profiles,).
    threshold : float, optional
        Potential density difference threshold [kg/m^3]. Default is 0.03.
    ref_depth : float, optional
        Reference depth [m] near the surface for baseline density. Default is 10.

    Returns
    -------
    mld : ndarray
        Mixed Layer Depths [m] for each profile, shape (n_profiles,).
        np.nan if MLD not found.
    """

    SA = np.asarray(SA)
    pt = np.asarray(pt)
    depth = np.asarray(depth)

    if SA.shape != pt.shape:
        raise ValueError("SA and pt must have the same shape (n_depths, n_profiles)")

    n_depths, n_profiles = SA.shape
    mld = np.full(n_profiles, np.nan)

    # Broadcast latitude if it's a scalar
    if np.isscalar(lat):
        lat = np.full(n_profiles, lat)
    lat = np.asarray(lat)

    for i in range(n_profiles):
        # Pressure from depth
        p = gsw.p_from_z(-depth, lat[i])

        # Potential density anomaly referenced to 0 dbar
        sigma0 = gsw.sigma0(SA[:, i], pt[:, i])  # shape: (n_depths,)

        # Reference density at ref_depth
        idx_ref = np.argmin(np.abs(depth - ref_depth))
        rho_ref = np.mean(sigma0[max(0, idx_ref - 1):idx_ref + 2])

        # Find first depth where density exceeds reference by threshold
        exceed = np.where(sigma0 > rho_ref + threshold)[0]
        if exceed.size > 0:
            mld[i] = depth[exceed[0]]

    return mld

In [ ]:
SA = np.array([
    [34.5, 34.4, 34.3],
    [34.6, 34.5, 34.4],
    [34.7, 34.6, 34.5],
    [34.8, 34.8, 34.6],
    [34.9, 34.9, 34.7]
])

pt = np.array([
    [18.0, 18.1, 18.2],
    [17.9, 18.0, 18.1],
    [17.5, 17.8, 18.0],
    [16.0, 17.5, 17.8],
    [14.0, 16.0, 17.0]
])

depth = np.array([0, 5, 10, 20, 30])  # m
lat = 30.0

mld = compute_mld_profiles(SA, pt, depth, lat)
print("Mixed Layer Depths (m):", mld)

import matplotlib.pyplot as plt

plt.plot(pt,-depth,'.-')
plt.show()

plt.plot(SA, -depth,'.-')
plt.show()